# Collaboration and Competition

---

In this notebook, you will learn how to use the Unity ML-Agents environment for the third project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893) program.

### 1. Start the Environment

We begin by importing the necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [1]:
from unityagents import UnityEnvironment
import numpy as np

Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Tennis.app"`
- **Windows** (x86): `"path/to/Tennis_Windows_x86/Tennis.exe"`
- **Windows** (x86_64): `"path/to/Tennis_Windows_x86_64/Tennis.exe"`
- **Linux** (x86): `"path/to/Tennis_Linux/Tennis.x86"`
- **Linux** (x86_64): `"path/to/Tennis_Linux/Tennis.x86_64"`
- **Linux** (x86, headless): `"path/to/Tennis_Linux_NoVis/Tennis.x86"`
- **Linux** (x86_64, headless): `"path/to/Tennis_Linux_NoVis/Tennis.x86_64"`

For instance, if you are using a Mac, then you downloaded `Tennis.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Tennis.app")
```

In [2]:
env = UnityEnvironment(file_name="Tennis.app")

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: TennisBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 8
        Number of stacked Vector Observation: 3
        Vector Action space type: continuous
        Vector Action space size (per agent): 2
        Vector Action descriptions: , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [3]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

In this environment, two agents control rackets to bounce a ball over a net. If an agent hits the ball over the net, it receives a reward of +0.1.  If an agent lets a ball hit the ground or hits the ball out of bounds, it receives a reward of -0.01.  Thus, the goal of each agent is to keep the ball in play.

The observation space consists of 8 variables corresponding to the position and velocity of the ball and racket. Two continuous actions are available, corresponding to movement toward (or away from) the net, and jumping. 

Run the code cell below to print some information about the environment.

In [4]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents 
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

Number of agents: 2
Size of each action: 2
There are 2 agents. Each observes a state with length: 24
The state for the first agent looks like: [ 0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.         -6.65278625 -1.5
 -0.          0.          6.83172083  6.         -0.          0.        ]


In [5]:
env_info = env.reset(train_mode=True)[brain_name]

### 3. Training 

In [6]:
import torch
from model import Agent
from collections import deque
import matplotlib.pyplot as plt

In [7]:
def ddpg(agent0, agent1, train=True, n_episodes=600, print_every=100):
        
    scores_deque = deque(maxlen=print_every)
    scores = []

    for i_episode in range(1, n_episodes+1):
        env_info = env.reset(train_mode=train)[brain_name]
        states = env_info.vector_observations
        score = 0
        agent0.reset()
        agent1.reset()
        
        while True:
            action0 = agent0.act(states[0]) 
            action1 = agent1.act(states[1])
            actions = np.concatenate((action0, action1), axis=0)
            
            env_info = env.step(actions)[brain_name]  
            next_states = env_info.vector_observations
            rewards = env_info.rewards
            done = env_info.local_done
            
            agent0.step(states[0], action0, rewards[0], next_states[0], done[0])
            agent1.step(states[1], action1, rewards[1], next_states[1], done[1])
            
            states = next_states
            score += np.max(rewards)
            if np.any(done):
                break 
                
        scores_deque.append(score)
        scores.append(score)
        print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_deque)), end="")
        
        if train:
            torch.save(agent0.actor_local.state_dict(), 'checkpoint_actor_agent0.pth')
            torch.save(agent0.critic_local.state_dict(), 'checkpoint_critic_agent0.pth')
            torch.save(agent1.actor_local.state_dict(), 'checkpoint_actor_agent1.pth')
            torch.save(agent1.critic_local.state_dict(), 'checkpoint_critic_agent1.pth')
            
        if i_episode % print_every == 0:
            print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_deque)))
            
            fig, ax = plt.subplots(1, 1, figsize=[15, 5])
            plt.rcParams.update({'font.size': 14})

            ax.plot(np.arange(len(scores)), scores, c='black')
            ax.set_ylabel('Score')
            ax.set_xlabel('Episode #')
            ax.grid(which="major")

            fig.tight_layout()
            fig.savefig("checkpoint_scores.jpg")
            
    return scores

In [8]:
agent0 = Agent(state_size=state_size, action_size=action_size, random_seed=2)
agent1 = Agent(state_size=state_size, action_size=action_size, random_seed=2, memory=agent0.memory)

scores = ddpg(agent0, agent1, train=True)

Episode 100	Average Score: 0.00
Episode 200	Average Score: 0.01
Episode 300	Average Score: 0.03
Episode 400	Average Score: 0.10
Episode 500	Average Score: 0.43
Episode 570	Average Score: 1.17

KeyboardInterrupt: 